In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('ODI_Match_Data.csv')

C:\Users\DJ\AppData\Local\Temp\ipykernel_22060\3708228576.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('ODI_Match_Data.csv')


In [3]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'cricsheet_id'],
      dtype='object')

In [4]:
df.drop(columns=['wicket_type','noballs','byes','wides','legbyes','penalty','start_date','season','other_player_dismissed','venue','other_wicket_type','striker','non_striker','bowler'], inplace=True)

In [5]:
teams = ['Australia','New Zeland','England', 'South Africa',
       'West Indies', 'Sri Lanka', 'Pakistan', 'India', 'Bangladesh', 'Afghanistan']

In [6]:
df = df[df['batting_team'].isin(teams)]
df = df[df['bowling_team'].isin(teams)]

In [7]:
df['runs'] = df['runs_off_bat']+df['extras']
df.drop(columns=['extras'], inplace=True)

In [8]:
df['current_score'] = df.groupby(['match_id','innings'])['runs'].cumsum()

In [9]:
df['over'] = df['ball'].apply(lambda x:str(x).split('.')[0])
df['ball_no'] = df['ball'].apply(lambda x:str(x).split('.')[1])

In [10]:
df['ball_bowled'] = (df['over'].astype(int)*6) + df['ball_no'].astype(int)
df['ball_left'] = 300-df['ball_bowled']
df['ball_left'] = df['ball_left'].apply(lambda x:0 if x<0 else x)

In [11]:
df['player_dismissed'] = df['player_dismissed'].fillna('0')
df['player_dismissed'] = df['player_dismissed'].apply(lambda x:0 if x=='0' else 1)
df['p_d_2'] = df['player_dismissed'].copy()
df['player_dismissed'] = df['player_dismissed'].astype(int)
df['player_dismissed'] = df.groupby(['match_id','innings'])['player_dismissed'].cumsum()
df['wickets_left'] = 10 - df['player_dismissed']

In [12]:
df['wickets_left'].describe()

count    689969.000000
mean          7.092951
std           2.309771
min           0.000000
25%           6.000000
50%           7.000000
75%           9.000000
max          10.000000
Name: wickets_left, dtype: float64

In [13]:
df['crr'] = (df['current_score']*6)/df['ball_bowled']

In [14]:
df['innings'].describe()

count    689969.000000
mean          1.460345
std           0.498425
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max           2.000000
Name: innings, dtype: float64

In [15]:
groups = df.groupby(['match_id', 'innings'])

last_ten = []
for (match_id, inning), group in groups:
    last_ten.extend(group['runs'].rolling(window=60).sum().tolist())

df['last_ten'] = last_ten

In [16]:
df = df.groupby(['match_id', 'innings'])['runs'].sum().reset_index().merge(df, on=['match_id','innings'])

In [17]:
df['dot_balls'] = df['runs_y'].apply(lambda x:1 if x==0 else 0)
df['boundary'] = df['runs_off_bat'].apply(lambda x:1 if x==4 or x==6 else 0)

In [18]:
groups = df.groupby(['match_id', 'innings'])

last_ten_dot = []
last_ten_boundary = []
last_ten_wkts = []
for (match_id, inning), group in groups:
    last_ten_dot.extend(group['dot_balls'].rolling(window=60).sum().tolist())
    last_ten_boundary.extend(group['boundary'].rolling(window=60).sum().tolist())
    last_ten_wkts.extend(group['p_d_2'].rolling(window=60).sum().tolist())

In [19]:
df['last_ten_dot'] = last_ten_dot
df['last_ten_boundary'] = last_ten_boundary
df['last_ten_wkts'] = last_ten_wkts

In [20]:
df=df[['innings','batting_team','bowling_team','current_score','ball_bowled','crr','ball_left','wickets_left','last_ten','last_ten_dot','last_ten_boundary','last_ten_wkts','runs_x']]
df.dropna(inplace=True)

C:\Users\DJ\AppData\Local\Temp\ipykernel_22060\485427035.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


In [21]:
df.head()

,innings,batting_team,bowling_team,current_score,ball_bowled,crr,ball_left,wickets_left,last_ten,last_ten_dot,last_ten_boundary,last_ten_wkts,runs_x
59,1,Australia,West Indies,47,54,5.222222,246,8,31.0,33.0,5.0,2.0,270
60,1,Australia,West Indies,47,55,5.127273,245,8,31.0,33.0,5.0,2.0,270
61,1,Australia,West Indies,48,56,5.142857,244,8,30.0,32.0,5.0,2.0,270
62,1,Australia,West Indies,48,57,5.052632,243,8,31.0,33.0,5.0,2.0,270
63,1,Australia,West Indies,50,58,5.172414,242,8,30.0,33.0,5.0,2.0,270


In [22]:
df.corr(numeric_only=True)['runs_x']

innings             -0.235931
current_score        0.435845
ball_bowled          0.176432
crr                  0.596651
ball_left           -0.176429
wickets_left         0.280142
last_ten            -0.035205
last_ten_dot        -0.524111
last_ten_boundary    0.339835
last_ten_wkts       -0.228268
runs_x               1.000000
Name: runs_x, dtype: float64

In [23]:
df.describe()

,innings,current_score,ball_bowled,crr,ball_left,wickets_left,last_ten,last_ten_dot,last_ten_boundary,last_ten_wkts,runs_x
count,425944.000000,425944.000000,425944.000000,425944.000000,425944.000000,425944.000000,425944.000000,425944.000000,425944.000000,425944.000000,425944.000000
mean,1.451881,139.103955,169.633886,4.872503,130.366623,6.440098,50.135577,30.047462,4.811623,1.355051,246.450733
std,0.497680,66.449152,67.272752,1.082916,67.271760,2.196893,15.070068,7.649626,2.883729,1.162630,60.267883
min,1.000000,7.000000,50.000000,0.608696,0.000000,0.000000,6.000000,3.000000,0.000000,0.000000,43.000000
25%,1.000000,85.000000,111.000000,4.139241,75.000000,5.000000,40.000000,25.000000,3.000000,0.000000,209.000000
50%,1.000000,131.000000,168.000000,4.813953,132.000000,7.000000,49.000000,30.000000,4.000000,1.000000,248.000000
75%,2.000000,184.000000,225.000000,5.536481,189.000000,8.000000,59.000000,35.000000,6.000000,2.000000,287.000000
max,2.000000,460.000000,303.000000,13.180328,250.000000,10.000000,181.000000,56.000000,28.000000,9.000000,481.000000


In [24]:
df.isnull().sum()

innings              0
batting_team         0
bowling_team         0
current_score        0
ball_bowled          0
crr                  0
ball_left            0
wickets_left         0
last_ten             0
last_ten_dot         0
last_ten_boundary    0
last_ten_wkts        0
runs_x               0
dtype: int64

In [ ]:
df.to_csv('final_dataset.csv', index=False)